<a href="https://colab.research.google.com/github/enjileona/Bootcamp-Data-Science-and-Mechine-Learning/blob/main/EXERCISE_Big%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK
1.   **Spark Batch Processing Dasar**
* Baca file penjaminan.csv dengan Spark.
* Hitung total & rata-rata jumlah per jenis_kredit.
*	Simpan hasilnya ke format Parquet

2.   **Data Transformation**
* Tambahkan kolom flag_kredit_besar (1 jika jumlah > 100 juta, else 0).
* Filter data dengan tenor >= 24 & status_pembayaran = "Menunggak".
*	Simpan hasil filter ke CSV.

3. **Fraud Detection Rule-based**
* Terapkan rule sederhana: jumlah > 400 juta dan tenor >= 48 & status_pembayaran = Menunggak
* Gabungkan hasil filter jadi fraud_candidates.
*	Hitung distribusi suspicious cases per jenis_kredit.

4. **Data Quality Check**
*	Cek jumlah record dengan NULL di kolom penting (jumlah, tenor, status_pembayaran).
*	Cek apakah ada duplikasi id_pengajuan.
*	Buat ringkasan hasil data quality check (tabel kecil).







# 1. Spark Batch Processing Dasar

In [ ]:
from pyspark.sql import functions as F

# Baca file CSV
df = spark.read.csv("penjaminan.csv", header=True, inferSchema=True)

# Lihat struktur data
df.printSchema()
df.show(5)

root
 |-- id_pengajuan: integer (nullable = true)
 |-- jenis_kredit: string (nullable = true)
 |-- jumlah: integer (nullable = true)
 |-- tenor: integer (nullable = true)
 |-- status_pembayaran: string (nullable = true)
 |-- tanggal: timestamp (nullable = true)

+------------+------------------+---------+-----+-----------------+-------------------+
|id_pengajuan|      jenis_kredit|   jumlah|tenor|status_pembayaran|            tanggal|
+------------+------------------+---------+-----+-----------------+-------------------+
|           1|  Kredit Konsumtif|318545614|    6|           Lancar|2021-01-01 00:00:00|
|           2|Kredit Modal Kerja|498899665|   60|           Lancar|2021-01-01 01:00:00|
|           3|  Kredit Konsumtif|275676561|   60|           Lancar|2021-01-01 02:00:00|
|           4|  Kredit Konsumtif|308310195|   24|      Gagal Bayar|2021-01-01 03:00:00|
|           5|Kredit Modal Kerja|149928360|   36|           Lancar|2021-01-01 04:00:00|
+------------+------------------+

In [ ]:
# Hitung total dan rata-rata jumlah per jenis_kredit
# Agregasi total jumlah kredit per jenis_kredit
print("\nAgregasi total jumlah kredit per jenis_kredit")
df.groupBy("jenis_kredit").agg(
   F.sum("jumlah").alias("total_kredit"),
   F.avg("jumlah").alias("rata_rata_kredit")
).show()




Agregasi total jumlah kredit per jenis_kredit
+------------------+------------+--------------------+
|      jenis_kredit|total_kredit|    rata_rata_kredit|
+------------------+------------+--------------------+
|  Kredit Konsumtif|829267661053|2.5136940316853592E8|
|Kredit Modal Kerja|844551306757| 2.518042059502087E8|
|  Kredit Investasi|833950411213|2.4916355279743055E8|
+------------------+------------+--------------------+



In [ ]:
# Save the DataFrame to a parquet file
df.write.parquet("penjaminan.parquet")

# 2. Data Transformation

In [ ]:
from pyspark.sql import functions as F

# Tambah Kolom flag_kredit_besar (1 jika jumlah > 100 juta, else 0)
print("\nMenambahkan kolom flag_kredit_besar")
df2 = df.withColumn("flag_kredit_besar", F.when(F.col("jumlah") > 100_000_000, 1).otherwise(0))
df2.select("jenis_kredit", "jumlah", "flag_kredit_besar").show(5)

# Filter data dengan tenor >= 24 dan status_pembayaran = "Menunggak"
print("\nFilter lebih kompleks")
df2.filter(
   (F.col("tenor") >= 24) &
   (F.col("status_pembayaran") == "Menunggak")
).select("id_pengajuan", "jenis_kredit", "tenor", "status_pembayaran").show(5)


Menambahkan kolom flag_kredit_besar
+------------------+---------+-----------------+
|      jenis_kredit|   jumlah|flag_kredit_besar|
+------------------+---------+-----------------+
|  Kredit Konsumtif|318545614|                1|
|Kredit Modal Kerja|498899665|                1|
|  Kredit Konsumtif|275676561|                1|
|  Kredit Konsumtif|308310195|                1|
|Kredit Modal Kerja|149928360|                1|
+------------------+---------+-----------------+
only showing top 5 rows


Filter lebih kompleks
+------------+------------------+-----+-----------------+
|id_pengajuan|      jenis_kredit|tenor|status_pembayaran|
+------------+------------------+-----+-----------------+
|          10|  Kredit Konsumtif|   60|        Menunggak|
|          15|  Kredit Investasi|   24|        Menunggak|
|          25|Kredit Modal Kerja|   24|        Menunggak|
|          29|  Kredit Konsumtif|   60|        Menunggak|
|          32|  Kredit Konsumtif|   60|        Menunggak|
+---------

In [ ]:
# Simpan hasil filter CSV file
df2.filter(
   (F.col("tenor") >= 24) &
   (F.col("status_pembayaran") == "Menunggak")
).write.csv("filtered_penjaminan.csv", header=True)

# 3. Fraud Detections Rule-Based

In [ ]:
# Terapkan rule sederhana
# 1. Jumlah > 400 Juta
# 2. Tenor => 48 dan status_pembayaran = Menunggak
fraud_candidates = df.filter(
   (F.col("jumlah") > 400_000_000) |
   ((F.col("tenor") >= 48) & (F.col("status_pembayaran") == "Menunggak")) |
   (F.col("id_pengajuan") % 999 == 0)
)

print("\nDataFrame fraud candidates:")
fraud_candidates.select("id_pengajuan", "jenis_kredit", "jumlah", "tenor", "status_pembayaran").show(10)

#Gabungkan hasil filter jadi fraud_candidates
print("\nJumlah suspicious cases:", fraud_candidates.count())

#Hitung distribusi suspicious cases per jenis_kredit
print("\nDistribusi suspicious cases per jenis kredit")
fraud_candidates.groupBy("jenis_kredit").count().show()


DataFrame fraud candidates:
+------------+------------------+---------+-----+-----------------+
|id_pengajuan|      jenis_kredit|   jumlah|tenor|status_pembayaran|
+------------+------------------+---------+-----+-----------------+
|           2|Kredit Modal Kerja|498899665|   60|           Lancar|
|           8|  Kredit Investasi|496214695|   12|           Lancar|
|          10|  Kredit Konsumtif|189548321|   60|        Menunggak|
|          17|  Kredit Investasi|453460932|   24|           Lancar|
|          21|Kredit Modal Kerja|473154743|    6|           Lancar|
|          24|  Kredit Investasi|420403905|   36|           Lancar|
|          29|  Kredit Konsumtif| 34159511|   60|        Menunggak|
|          32|  Kredit Konsumtif|252560587|   60|        Menunggak|
|          38|  Kredit Konsumtif|415714976|   36|        Menunggak|
|          39|Kredit Modal Kerja| 96762455|   60|        Menunggak|
+------------+------------------+---------+-----+-----------------+
only showing top 10

# 4. Data Quality Check

In [ ]:
#Cek jumlah record dengan Null di kolom penting (jumlah, tenor, status_pembayaran)

#Cek apakah ada duplikasi id_pengajuan

#Buat ringkasan hasil data quality check (tabel kecil)

In [ ]:
# Cek jumlah record dengan Null di kolom penting (jumlah, tenor, status_pembayaran)
print("\nJumlah record dengan Null di kolom:")
df.select(
   F.sum(F.when(F.col("jumlah").isNull(), 1).otherwise(0)).alias("null_jumlah"),
   F.sum(F.when(F.col("tenor").isNull(), 1).otherwise(0)).alias("null_tenor"),
   F.sum(F.when(F.col("status_pembayaran").isNull(), 1).otherwise(0)).alias("null_status_pembayaran")
).show()

#Cek apakah ada duplikasi id_pengajuan
print("\nJumlah record duplikat berdasarkan id_pengajuan:")
df.groupBy("id_pengajuan").count().filter("count > 1").show()

#Buat ringkasan hasil data quality check (tabel kecil)



Jumlah record dengan Null di kolom:
+-----------+----------+----------------------+
|null_jumlah|null_tenor|null_status_pembayaran|
+-----------+----------+----------------------+
|          0|         0|                     0|
+-----------+----------+----------------------+


Jumlah record duplikat berdasarkan id_pengajuan:
+------------+-----+
|id_pengajuan|count|
+------------+-----+
+------------+-----+

